In [15]:
from reader import Reader
from dataloader import DatasetTrain, DatasetTest, NewsDataset
from torch.utils.data import DataLoader
import pandas as pd
import torch
import os
import numpy as np
from argument import parse_naml_args
from log import dump_args, setuplogger, get_checkpoint
import random
import logging
import torch.optim as optim
from model.naml import Model
setuplogger()
args = parse_naml_args()
dump_args(args)
random.seed(args.seed)
torch.manual_seed(args.seed)



reader = Reader(args)
logging.info('Preparing training data...')

[INFO 2025-02-08 20:51:24,970] args[batch_size]=32
[INFO 2025-02-08 20:51:24,970] args[batch_size]=32
[INFO 2025-02-08 20:51:24,971] args[category_emb_dim]=100
[INFO 2025-02-08 20:51:24,971] args[category_emb_dim]=100
[INFO 2025-02-08 20:51:24,973] args[custom_abstract_dir]=
[INFO 2025-02-08 20:51:24,973] args[custom_abstract_dir]=
[INFO 2025-02-08 20:51:24,974] args[drop_rate]=0.2
[INFO 2025-02-08 20:51:24,974] args[drop_rate]=0.2
[INFO 2025-02-08 20:51:24,976] args[enable_gpu]=True
[INFO 2025-02-08 20:51:24,976] args[enable_gpu]=True
[INFO 2025-02-08 20:51:24,976] args[epochs]=5
[INFO 2025-02-08 20:51:24,976] args[epochs]=5
[INFO 2025-02-08 20:51:24,977] args[filter_num]=3
[INFO 2025-02-08 20:51:24,977] args[filter_num]=3
[INFO 2025-02-08 20:51:24,978] args[freeze_embedding]=False
[INFO 2025-02-08 20:51:24,978] args[freeze_embedding]=False
[INFO 2025-02-08 20:51:24,979] args[glove_embedding_path]=/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader/data/glove.840B.300d.txt
[INFO 2025-02

In [2]:
if (args.use_custom_abstract):
      custom_abstract_df = pd.read_csv(args.custom_abstract_dir)
      custom_abstract_dict = custom_abstract_df.set_index('news_id')['abstract'].to_dict()
      news, news_index, category_dict, subcategory_dict, word_cnt = reader.read_custom_abstract(
          os.path.join(args.train_data_dir, 'news.tsv'), custom_abstract_dict)
else:    
      news, news_index, category_dict, subcategory_dict, word_dict = reader.read_news(
      os.path.join(args.train_data_dir, 'news.tsv'), args, mode='train')
num_training_behaviors = reader.prepare_training_data(args.train_data_dir)
num_testing_behaviors = reader.prepare_testing_data(args.test_data_dir)



51282it [00:03, 16823.75it/s]
156965it [00:02, 69104.06it/s]


[INFO 2025-02-08 19:57:50,159] Writing files...


73152it [00:00, 1073430.03it/s]

[INFO 2025-02-08 19:57:50,487] Writing files...


In [3]:
news_title, news_category, news_subcategory, news_abstract = reader.get_doc_input(
        news, news_index, category_dict, subcategory_dict, word_dict, args)
news_combined = np.concatenate([x for x in [news_title, news_category, news_subcategory, news_abstract] if x is not None], axis=-1)


100%|██████████| 51282/51282 [00:00<00:00, 63255.84it/s]


In [4]:
news_combined

array([[    0,     0,     0, ...,     0,     0,     0],
       [    1,     2,     3, ...,  1770,  1434,     0],
       [   12,    13,    14, ...,     0,  7578,    46],
       ...,
       [  586,     0,  3536, ...,     0,  3215, 10852],
       [   33,     1,  7140, ...,    28,  1915,     0],
       [   76,  1137,   407, ...,     0,     0,     0]], dtype=int32)

In [5]:

logging.info('Initializing word embedding matrix...')

embedding_matrix, have_word = reader.load_matrix(args.glove_embedding_path,
                                            word_dict,
                                            args.word_embedding_dim)
logging.info(f'Word dict length: {len(word_dict)}')
logging.info(f'Have words: {len(have_word)}')
logging.info(f'Missing rate: {(len(word_dict) - len(have_word)) / len(word_dict)}')

model = Model(args, embedding_matrix, len(category_dict), len(subcategory_dict))


[INFO 2025-02-08 19:57:51,408] Initializing word embedding matrix...
[INFO 2025-02-08 19:58:26,051] Word dict length: 12519
[INFO 2025-02-08 19:58:26,052] Have words: 11960
[INFO 2025-02-08 19:58:26,053] Missing rate: 0.0446521287642783


In [7]:
if args.load_ckpt_name is not None:
        ckpt_path = get_checkpoint(args.model_dir, args.load_ckpt_name)
        checkpoint = torch.load(ckpt_path, map_location='cpu')
        model.load_state_dict(checkpoint['model_state_dict'])
        logging.info(f"Model loaded from {ckpt_path}.")

optimizer = optim.Adam(model.parameters(), lr=args.lr)

# if args.enable_gpu:
#     model = model.cuda(0)

# if rank == 0:
#     print(model)
#     for name, param in model.named_parameters():
#         print(name, param.requires_grad)

data_file_path = os.path.join(args.train_data_dir, f'behaviors_np{args.npratio}_{0}.tsv')

dataset = DatasetTrain(data_file_path, news_index, news_combined, args)
dataloader = DataLoader(dataset, batch_size=args.batch_size)

In [ ]:
import sys
sys.path.append('/home/vinmike/Documents/GitHub/LLM4Rec-Dataloader') 

# Import the metrics
from metric import acc, auc_score, ndcg_score, mrr_score, ctr_score

In [14]:

rank = 0
logging.info('Training...')
for ep in range(args.start_epoch, args.epochs):
    loss = 0.0
    accuary = 0.0
    for cnt, (log_ids, log_mask, input_ids, targets) in enumerate(dataloader):
        # if args.enable_gpu:
        #     log_ids = log_ids.cuda(rank, non_blocking=True)
        #     log_mask = log_mask.cuda(rank, non_blocking=True)
        #     input_ids = input_ids.cuda(rank, non_blocking=True)
        #     targets = targets.cuda(rank, non_blocking=True)

        bz_loss, y_hat = model(log_ids, log_mask, input_ids, targets)
        loss += bz_loss.data.float()
        accuary += acc(targets, y_hat)
        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()

        if cnt % args.log_steps == 0:
            logging.info(
                '[{}] Ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(
                    rank, cnt * args.batch_size, loss.data / cnt, accuary / cnt)
            )

        if rank == 0 and     cnt != 0 and cnt % args.save_steps == 0:
            ckpt_path = os.path.join(args.model_dir, f'epoch-{ep+1}-{cnt}.pt')
            torch.save(
                {
                    'model_state_dict': model.state_dict(),
                    'category_dict': category_dict,
                    'word_dict': word_dict,
                    'subcategory_dict': subcategory_dict
                }, ckpt_path)
            logging.info(f"Model saved to {ckpt_path}.")

    logging.info('Training finish.')

    if rank == 0:
        ckpt_path = os.path.join(args.model_dir, f'epoch-{ep+1}.pt')
        torch.save(
            {
                'model_state_dict':model.state_dict(),
                'category_dict': category_dict,
                'subcategory_dict': subcategory_dict,
                'word_dict': word_dict,
            }, ckpt_path)
        logging.info(f"Model saved to {ckpt_path}.")



[INFO 2025-02-08 20:16:34,443] Training...
[INFO 2025-02-08 20:16:37,812] [0] Ed: 0, train_loss: inf, acc: inf
[INFO 2025-02-08 20:24:07,513] [0] Ed: 3200, train_loss: 1.59379, acc: 0.34156
[INFO 2025-02-08 20:32:05,838] [0] Ed: 6400, train_loss: 1.54577, acc: 0.35703
[INFO 2025-02-08 20:40:10,140] [0] Ed: 9600, train_loss: 1.52555, acc: 0.36542


KeyboardInterrupt: 

Test


In [17]:
args.mode = 'test'
args.user_log_mask=True
args.batch_size=128
args.load_ckpt_name= 'epoch-5.pt'
args.prepare=True
total_sample_num = reader.prepare_testing_data(args.test_data_dir)


73152it [00:00, 657307.04it/s]

[INFO 2025-02-08 20:52:52,304] Writing files...
[INFO 2025-02-08 20:52:52,304] Writing files...


In [19]:
import tqdm

# torch.cuda.set_device(rank)

if args.load_ckpt_name is not None:
    ckpt_path = get_checkpoint(args.model_dir, args.load_ckpt_name)

assert ckpt_path is not None, 'No checkpoint found.'
checkpoint = torch.load(ckpt_path, map_location='cpu')

subcategory_dict = checkpoint['subcategory_dict']
category_dict = checkpoint['category_dict']
word_dict = checkpoint['word_dict']

dummy_embedding_matrix = np.zeros((len(word_dict) + 1, args.word_embedding_dim))
model = Model(args, dummy_embedding_matrix, len(category_dict), len(subcategory_dict))
model.load_state_dict(checkpoint['model_state_dict'])
logging.info(f"Model loaded from {ckpt_path}")

if args.enable_gpu:
    model.cuda(rank)

model.eval()
torch.set_grad_enabled(False)

if (args.use_custom_abstract):
    custom_abstract_df = pd.read_csv(args.custom_abstract_dir)
    custom_abstract_dict = custom_abstract_df.set_index('news_id')['abstract'].to_dict()
    news, news_index, category_dict, subcategory_dict, word_cnt = reader.read_custom_abstract(
        os.path.join(args.train_data_dir, 'news.tsv'), custom_abstract_dict)
else:
    news, news_index, category_dict, subcategory_dict, word_dict = reader.read_news(
        os.path.join(args.train_data_dir, 'news.tsv'), args, mode='train')
news_title, news_category, news_subcategory, news_abstract = reader.get_doc_input(
    news, news_index, category_dict, subcategory_dict, word_dict, args)
news_combined = np.concatenate([x for x in [news_title, news_category, news_subcategory, news_abstract] if x is not None], axis=-1)

news_dataset = NewsDataset(news_combined)
news_dataloader = DataLoader(news_dataset,
                                batch_size=args.batch_size,
                                num_workers=4)

news_scoring = []
with torch.no_grad():
    for input_ids in tqdm(news_dataloader):
        input_ids = input_ids.cuda(rank)
        news_vec = model.news_encoder(input_ids)
        news_vec = news_vec.to(torch.device("cpu")).detach().numpy()
        news_scoring.extend(news_vec)

news_scoring = np.array(news_scoring)
logging.info("news scoring num: {}".format(news_scoring.shape[0]))

if rank == 0:
    doc_sim = 0
    for _ in tqdm(range(1000000)):
        i = random.randrange(1, len(news_scoring))
        j = random.randrange(1, len(news_scoring))
        if i != j:
            doc_sim += np.dot(news_scoring[i], news_scoring[j]) / (np.linalg.norm(news_scoring[i]) * np.linalg.norm(news_scoring[j]))
    logging.info(f'News doc-sim: {doc_sim / 1000000}')

data_file_path = os.path.join(args.test_data_dir, f'behaviors_{rank}.tsv')

def collate_fn(tuple_list):
    log_vecs = torch.FloatTensor([x[0] for x in tuple_list])
    log_mask = torch.FloatTensor([x[1] for x in tuple_list])
    news_vecs = [x[2] for x in tuple_list]
    labels = [x[3] for x in tuple_list]
    return (log_vecs, log_mask, news_vecs, labels)

dataset = DatasetTest(data_file_path, news_index, news_scoring, args)
dataloader = DataLoader(dataset, batch_size=args.batch_size, collate_fn=collate_fn)

AUC = []
MRR = []
nDCG5 = []
nDCG10 = []

def print_metrics(rank, cnt, x):
    logging.info("[{}] {} samples: {}".format(rank, cnt, '\t'.join(["{:0.2f}".format(i * 100) for i in x])))

def get_mean(arr):
    return [np.array(i).mean() for i in arr]

def get_sum(arr):
    return [np.array(i).sum() for i in arr]

local_sample_num = 0

for cnt, (log_vecs, log_mask, news_vecs, labels) in enumerate(dataloader):
    local_sample_num += log_vecs.shape[0]

    if args.enable_gpu:
        log_vecs = log_vecs.cuda(rank, non_blocking=True)
        log_mask = log_mask.cuda(rank, non_blocking=True)

    user_vecs = model.user_encoder(log_vecs, log_mask).to(torch.device("cpu")).detach().numpy()

    for user_vec, news_vec, label in zip(user_vecs, news_vecs, labels):
        if label.mean() == 0 or label.mean() == 1:
            continue

        score = np.dot(news_vec, user_vec)

        auc = auc_score(label, score)
        mrr = mrr_score(label, score)
        ndcg5 = ndcg_score(label, score, k=5)
        ndcg10 = ndcg_score(label, score, k=10)

        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)

    if cnt % args.log_steps == 0:
        print_metrics(rank, local_sample_num, get_mean([AUC, MRR, nDCG5, nDCG10]))

logging.info('[{}] local_sample_num: {}'.format(rank, local_sample_num))

print_metrics('*', local_sample_num, get_mean([AUC, MRR, nDCG5, nDCG10]))


AssertionError: No checkpoint found.